In [ ]:
#!/usr/bin/python

import numpy as np
import pyfits
import os
from matplotlib import pyplot as plt
import delta_r_utils as utils
import environment_utils as envutils

%matplotlib inline
from matplotlib import pyplot as plt
from matplotlib.colors import LogNorm
from matplotlib import gridspec
from matplotlib import rc, rcParams

inf = np.inf

# Make use of TeX
rc('text',usetex=True)

# Change all fonts to 'Computer Modern'
rc('font',**{'family':'serif','serif':['Computer Modern']})


centering = 'BCG'
envnames = ['Void', 'Sheet', 'Filament', 'Knot']
envcolors = ['red', 'green', 'blue', 'orange']

path_gamacats = '/data2/brouwer/MergedCatalogues'
gamacatname = '%s/GAMACatalogue_1.0.fits'%path_gamacats
shuffledcatname = '%s/shuffled_environment_S4_deltaR.fits'%path_gamacats
path_results = 'results'

rankmin = -999
rankmax = inf

shuffled = True

In [ ]:
# Importing GAMA catalogue
print 'Importing GAMA catalogue:', gamacatname
gamacat = pyfits.open(gamacatname)[1].data
shuffledcat = pyfits.open(shuffledcatname)[1].data

galIDlist = gamacat['ID']

# Importing angular seperation
angseplist = gamacat['AngSep%s'%centering]
angseplist[angseplist<=0] = 0.

# Importing and correcting log(Mstar)
logmstarlist = gamacat['logmstar']
fluxscalelist = gamacat['fluxscale'] # Fluxscale, needed for stellar mass correction
corr_list = np.log10(fluxscalelist)# - 2*np.log10(h/0.7)
logmstarlist = logmstarlist + corr_list
ranklist = gamacat['rankBCG']
nQlist = gamacat['nQ']
zlist =  gamacat['Z']

# Applying a mask to the galaxies
obsmask = (fluxscalelist<500)&(logmstarlist>5) & (0 <= gamacat['envS4'])&(gamacat['envS4'] < 4) & \
                                                    (rankmin <= ranklist)&(ranklist < rankmax) & (nQlist >= 3.)

#obsmask = (fluxscalelist<500)&(logmstarlist>5) & (rankmin <= ranklist)&
#obsmask = (0.039 < zlist)&(zlist <= 0.263)
#obsmask = (0 <= gamacat['envS4'])&(gamacat['envS4'] < 4)

logmstarlist = logmstarlist[obsmask]
mstarlist = 10**logmstarlist
angseplist = angseplist[obsmask]

zlist = gamacat['Z'][obsmask]
ranklist = gamacat['rank%s'%centering][obsmask]

if not shuffled:
    # Importing the real environment
    envlist = gamacat['envS4'][obsmask]
else:
    # Importing the shuffled environment
    envlist = shuffledcat['shuffenvR4'][obsmask]
    
print 'Imported: %i of %i galaxies (%g percent)'%(len(logmstarlist), len(galIDlist), \
                                    float(len(logmstarlist))/float(len(galIDlist))*100.)

In [ ]:
# Creating the log(Mstar) histogram (needed for the mstarweight)

nbins = 100
logmstarbins, logmstarhists, logmstarhistcens = \
envutils.create_histogram('log(Mstar)', logmstarlist, nbins, envnames, envlist, 'lin', False, False, False)

print 

In [ ]:
# Creating the logmstar weights for each lens

# Create the average histogram
avhist = np.mean(logmstarhists, 0)

# Calculate the weight for each logmstar bin
weighthists = avhist/logmstarhists
weighthists[np.logical_not(np.isfinite(weighthists))] = 0

#####################

# Show results
plt.figure(1, figsize=(8, 6))
#plt.plot(logmstarhistcens, avhist, '', ls='-', color='black', label='Average')

for env in xrange(len(envnames)):
    plt.plot(logmstarhistcens, logmstarhists[env], '', ls='-', color=envcolors[env], label=envnames[env])
    plt.axvline(x = np.median(logmstarlist[envlist==env]), color=envcolors[env])
    
plt.xlim(8, 12)
#plt.ylim(1e-3, 1e1)

plt.yscale('log')
plt.legend(fontsize=15)

xlabel = 'log(M$_*$)'
#ylabel = 'Number of galaxies'
ylabel = 'Normalized number of galaxies'


plt.ylabel(r'%s'%ylabel,fontsize=18)
plt.xlabel(r'%s'%xlabel,fontsize=18)
plt.tick_params(axis='both', which='major', labelsize=14)

histname = 'environment_logmstar_histogram'
if shuffled:
    histname = '%s_shuffled'%(histname)

plotname = '%s.png'%histname

plt.savefig(plotname, format='png')
print 'Written: ESD profile plot:', plotname
   
plt.show()


In [ ]:
# Calculate the logmstar weight for each galaxy
weightlist = np.zeros(len(logmstarlist)) # Length of the merged GAMA catalog

for env in xrange(len(envnames)): # For each environment
    envname = envnames[env]
    envmask = (envlist == env) # Mask the lenses outside that environment
    for b in xrange(nbins): # For each logmstar bin
        binmask = (logmstarbins[b]<=logmstarlist) & (logmstarlist<logmstarbins[b+1]) # Mask the lenses outside that bin
        totmask = envmask & binmask
        weight = weighthists[env,b]
        #print 'Env: %s, Bin: %i, Weight: %g'%(envname, b, weight)
        weightlist[totmask] = weight

In [ ]:
# Print the logmstarweight to a fits file
weightlist_tot = np.zeros(len(galIDlist))
weightlist_tot[obsmask] = weightlist


filename = '%s/mstarweight_rank%g-%g.fits'%(path_results, rankmin, rankmax)
if shuffled:
    filename = filename.replace('.fits', '_shuffled.fits')

envutils.write_catalog(filename, galIDlist, ['mstarweight'], [weightlist_tot])


In [ ]:
logmstarbins, logmstarhists, logmstarhistcens = envutils.create_histogram('log(Mstar)',logmstarlist, nbins, envnames, envlist, 'lin', False, weightlist, False)

In [ ]:



# Repeating AngSep and average quantity calculations for weighted galaxies

In [ ]:
# Calculating average redshift, log(M*) and satellite fraction of the lens samples (needed for halo model)

if shuffled:
    print 'For the shuffled environments:'
else:
    print 'For the cosmic environments:'
print 

print 'Without logmstarweight:'
zaverage, mstaraverage, fsat, fsatmax = envutils.calc_halomodel_input(envnames, envlist, ranklist, zlist, mstarlist, False)


print
print 'With logmstarweight:'
zaverage, mstaraverage, fsat, fsatmax = envutils.calc_halomodel_input(envnames, envlist, ranklist, zlist, mstarlist, weightlist)